<h1>Spark basics practice</h1>
<li>The code in the next cell extracts covid data from New York State's covid repository</li>
<li>The extracted data is stored in an RDD containing an Array of (String, String,Int,Int) matching (date, borough, positive cases, tests) for each day since March 1st 2020 (the data is ordered by time)</li>
<li>Use this RDD to answer the questions below</li>

In [1]:
val counties = Array("New+York", "Bronx","Kings","Queens","Richmond")
val base_url = "https://health.data.ny.gov/resource/xdss-u53e.json?County="
val urls = counties.map(a => base_url+a)
val results = urls.map(u => scala.io.Source.fromURL(u).mkString)
val data_rdd = spark.read.json(sc.parallelize(results)).rdd.map(r => (r(4).toString.slice(0,10), r(0).toString,r(3).toString.toInt,r(5).toString.toInt))

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.6:4043
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1646108905357)
SparkSession available as 'spark'


counties: Array[String] = Array(New+York, Bronx, Kings, Queens, Richmond)
base_url: String = https://health.data.ny.gov/resource/xdss-u53e.json?County=
urls: Array[String] = Array(https://health.data.ny.gov/resource/xdss-u53e.json?County=New+York, https://health.data.ny.gov/resource/xdss-u53e.json?County=Bronx, https://health.data.ny.gov/resource/xdss-u53e.json?County=Kings, https://health.data.ny.gov/resource/xdss-u53e.json?County=Queens, https://health.data.ny.gov/resource/xdss-u53e.json?County=Richmond)
results: Array[String] =
Array("[{"test_date":"2020-03-01T00:00:00.000","county":"New York","new_positives":"0","cumulative_number_of_positives":"0","total_number_of_tests":"0","cumulative_number_of_tests":"0"}
,{"test_date":"2020-03-02T00:00:00.000","county":"New York","new_positives...


<h1>Question 1</h1>
<li>Using <span style="color:blue">reduce</span> calculate the total number of cases and total number of tests in New York City</li>

In [2]:
val total_cases_tests = data_rdd
.map { case(a,b,c,d) => (b,(c,d) )} // gives nested tuple
.reduce((tuple1: (String, (Int, Int)), tuple2 :(String, (Int, Int))) => {
    
  val t1 = tuple1 match { case (i, t) => t }
  val t2 = tuple2 match { case (i, t) => t }
    
   ("New York",(t1._1 + t2._1, t1._2 + t2._2))}  
)
// the return tuple is in the form of  (signal of name (i.e.New York City),(total# cases, total# tests))

total_cases_tests: (String, (Int, Int)) = (New York,(2271826,48106799))


In [3]:
val total_cases_tests = data_rdd.map(t=>(t._3, t._4))
.reduce((a,b)=>(a._1+b._1, a._2+b._2))
//this one also works!

total_cases_tests: (Int, Int) = (2271826,48106799)


In [4]:
total_cases_tests

res0: (Int, Int) = (2271826,48106799)


<h1>Question 2</h1>
Using <span style="color:blue">reduceByKey</span> calculate the number of cases and total number of tests by borough

In [5]:
val cases_tests_by_borough =data_rdd
.map{ case(a,b, c,d) => (b,(c,d))}.reduceByKey((x,y)=>(x._1+y._1, x._2+y._2))
.collect

cases_tests_by_borough: Array[(String, (Int, Int))] = Array((Bronx,(392656,7101198)), (Richmond,(162015,3048360)), (New York,(402298,11588865)), (Kings,(680422,14584378)), (Queens,(634435,11783998)))


In [6]:
cases_tests_by_borough

res1: Array[(String, (Int, Int))] = Array((Bronx,(392656,7101198)), (Richmond,(162015,3048360)), (New York,(402298,11588865)), (Kings,(680422,14584378)), (Queens,(634435,11783998)))


<h1>Question 3</h1>
Using <span style="color:blue">reduceByKey</span> and <a href="http://homepage.cs.latrobe.edu.au/zhe/ZhenHeSparkRDDAPIExamples.html#sortBy">sortBy</a> calculate the number of cases and number of tests by month and return an RDD of Array(Month,(cases,tests)) sorted by the number of cases

In [7]:
val cases_tests_by_month = data_rdd
.map{ case(a,b, c,d) => (a.slice(0,7),(c,d))}.reduceByKey((x,y)=>(x._1+y._1, x._2+y._2))//find total cases and tests
.map{ case(a,b) => (a,b._1,b._2)}
.sortBy(t=>t._2,true)//order by number of cases
.map{ case(a,b,c) => (a,(b,c))}//return as the required form
.collect

cases_tests_by_month: Array[(String, (Int, Int))] = Array((2021-06,(6175,1551252)), (2020-08,(9082,1035847)), (2020-07,(9985,868254)), (2020-09,(10032,1050648)), (2020-06,(11711,816215)), (2021-05,(19254,2115327)), (2020-10,(19581,1494233)), (2021-07,(21784,1412611)), (2021-10,(31973,2740851)), (2020-05,(34074,543214)), (2021-11,(35769,2321502)), (2022-02,(39624,2120046)), (2020-03,(47439,96528)), (2021-09,(48212,2569733)), (2020-11,(50393,2008252)), (2021-08,(57130,2220115)), (2021-04,(78311,2700065)), (2021-02,(111431,2499763)), (2020-12,(116805,2567996)), (2021-03,(121579,2924050)), (2020-04,(122251,339464)), (2021-01,(172218,3054118)), (2021-12,(459493,4451205)), (2022-01,(637520,4605510)))


<h1>Question 4</h1>
Return an RDD of (date,borough,positivity) where positivity is the percentage of tests that are positive. For this problem, you must use the Option case class to handle the case where the divisor is zero

In [8]:
def zero_divisor(x:(String,String,Int,Int)): Option[(String,String,Double)] = {
    try {
        Some((x._1, x._2, 100.0*x._3/x._4))
    } catch {
        case e: Exception => None
    }
}

val positivity = data_rdd
.map(x => zero_divisor(x))
.flatMap(x=>x)

zero_divisor: (x: (String, String, Int, Int))Option[(String, String, Double)]
positivity: org.apache.spark.rdd.RDD[(String, String, Double)] = MapPartitionsRDD[28] at flatMap at <console>:34


In [9]:
positivity.collect

res2: Array[(String, String, Double)] = Array((2020-03-01,New York,NaN), (2020-03-02,New York,NaN), (2020-03-03,New York,0.0), (2020-03-04,New York,25.0), (2020-03-05,New York,5.555555555555555), (2020-03-06,New York,21.428571428571427), (2020-03-07,New York,0.0), (2020-03-08,New York,0.0), (2020-03-09,New York,14.893617021276595), (2020-03-10,New York,15.384615384615385), (2020-03-11,New York,4.285714285714286), (2020-03-12,New York,9.67741935483871), (2020-03-13,New York,14.552238805970148), (2020-03-14,New York,14.507772020725389), (2020-03-15,New York,18.92744479495268), (2020-03-16,New York,17.33668341708543), (2020-03-17,New York,26.923076923076923), (2020-03-18,New York,31.191806331471135), (2020-03-19,New York,35.38461538461539), (2020-03-20,New York,29.295328582739508), (2020-0...


<h1>Question 5</h1>
Return the tuple (date,borough,positivity) where the positivity was the highest (use <span style="color:blue">takeOrdered</span>)

In [10]:
val highest_positive = positivity
.takeOrdered(1)(Ordering[Double].on(-1*_._3))

highest_positive: Array[(String, String, Double)] = Array((2020-03-29,Queens,65.69190600522194))
